# 第 4 章 RAG 的基礎–用搜尋與檢索幫 AI 長知識

In [ ]:
from google.colab import userdata
from IPython.display import display, Markdown
from rich.pretty import pprint
import openai
import sys
import pickle

In [ ]:
client = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

## 4-1 使用內建搜尋工具幫 AI 走遍全世界

In [ ]:
response = client.responses.create(
    model="gpt-4.1-mini", # gpt-4.1-nano 不支援搜尋
    input="2024年十二強世界棒球賽冠軍是哪一隊？"
)
print(response.output_text)

使用網頁搜尋的價格可參考[這裡](https://platform.openai.com/docs/pricing#web-search)。雖然搜尋內容不會算入 tokens 數量計費，但搜尋是依次計費，沒有很便宜喔，自己使用 [Google Custom Search JSON API](https://developers.google.com/custom-search/v1/overview#pricing) 可能還便宜一些。

gpt-4o 以及 gpt-4.1 家族可用，不含 gpt-4.1-nano，之前測試 gpt-4o-mini 搜尋效果不佳

### 啟用內建的搜尋功能

In [ ]:
response = client.responses.create(
    model="gpt-4.1-mini", # gpt-4.1-nano 不支援搜尋
    input="十二強世界棒球賽冠軍是哪一隊？",
    tools=[{"type": "web_search_preview"}],
)

IPython 的 Markdown 好像解譯回覆中的格式會出錯？

In [ ]:
print(response.output_text)

In [ ]:
display(Markdown(response.output_text))

In [ ]:
pprint(response)

看不出來搜尋的關鍵字是什麼？

In [ ]:
from urllib.parse import unquote

def show_search_results(response):
    if response.output[0].type != "web_search_call": return
    content = response.output[1].content[0]
    for i, annotaion in enumerate(content.annotations, start=1):
        print(f'{i}. {annotaion.title}')
        print(f'   {unquote(annotaion.url)}')

show_search_results(response)

In [ ]:
pprint(response.usage)

### 設定搜尋地區

In [ ]:
response = client.responses.create(
    model="gpt-4.1-mini",
    input='推薦好吃的義大利餐廳',
    tools=[{"type": "web_search_preview"}],
)

In [ ]:
display(Markdown(response.output_text))

In [ ]:
pprint(response.tools[0].user_location)

- [ISO 國別碼](https://zh.wikipedia.org/zh-tw/ISO_3166-1)
- [IANA 時區](https://en.wikipedia.org/wiki/List_of_tz_database_time_zones)

In [ ]:
response = client.responses.create(
    model="gpt-4.1-mini",
    input='推薦好吃的義大利餐廳',
    tools=[
        {
            "type": "web_search_preview",
            "user_location": {
                "type": "approximate", # 固定內容
                "country": "TW", # ISO 國別碼
                "timezone": "Asia/Taipei", # IANA 時區
                "region": "台北", # 自由填寫
                "city": "台北"    # 自由填寫
            }
        }
    ],
)

In [ ]:
display(Markdown(response.output_text))

### 設定提供給模型的搜尋資料量

In [ ]:
response = client.responses.create(
    model="gpt-4.1-mini",
    input="十二強世界棒球賽冠軍是哪一隊？",
    tools=[{
        "type": "web_search_preview",
        # low, medium（default）, high
        "search_context_size": "high"
    }],
)
display(Markdown(response.output_text))

In [ ]:
show_search_results(response)

In [ ]:
pprint(response.usage)

### 串流方式使用工具

In [ ]:
response = client.responses.create(
    model="gpt-4.1-mini",
    input="十二強世界棒球賽冠軍是哪一隊？",
    tools=[{
        "type": "web_search_preview",
    }],
    stream=True
)

In [ ]:
for chunk in response:
    pprint(chunk)

搜尋結果在 type 為 response.output_text.annotation.added 的事件中

## 4-2 幫簡易聊天程式加上搜尋功能

### 設計處理指令的類別

In [ ]:
class BaseComand:
    def __init__(self, command, tool_name, icon):
        self.command = command      # 指令
        self.tool_name = tool_name  # 工具名稱
        self.icon = icon            # 工具圖示字元

    def handle_command(self, chat, cmd):
        # 不是正確的指令開頭（空字串會是 True）
        if not cmd.startswith(self.command):
            return False
        return True

### 修改 Chat 類別

In [ ]:
class Chat:
    def __init__(self, client, **kwargs):
        self._client = client
        self._last_id = kwargs.pop('last_id', None)
        self.tools = [] # 預設沒有使用工具
        self._commands = kwargs.pop('commands', [])

    def find_tool_index(self, tool_type):
        for i, tool in enumerate(self.tools):
            if tool['type'] == tool_type: return i
        return -1

    def _get_prompt(self):
        prompt = ''
        for command in self._commands:
            idx = self.find_tool_index(command.tool_name)
            if idx != -1:
                prompt += f'{command.icon}'
        prompt = f'({prompt})>>> ' if prompt else '>>> '
        return prompt

    def _process_command(self, cmd):
        for command in self._commands:
            if command.handle_command(self, cmd):
                return True
        return False

    def get_reply_text(self, msg, **kwargs) -> str:
        instructions = kwargs.pop('instructions', '使用繁體中文')
        model = kwargs.pop('model', 'gpt-4.1-nano')
        stream = kwargs.pop('stream', False)
        try:
            response = self._client.responses.create(
                instructions=instructions,
                model=model,
                input=msg,
                stream=True, # 都以串流方式處理，簡化程式邏輯
                previous_response_id=self._last_id, # 串接回應
                **kwargs
            )
            for event in response:
                if event.type == 'response.output_text.delta':
                    if stream: # 串流模式生成片段內容
                        yield event.delta
                elif event.type == 'response.completed':
                    self._last_id = event.response.id # 記錄識別碼
                    if not stream: # 非串流生成完整內容
                        yield event.response.output_text
        except openai.APIError as err:
            print(f'Error:{err.body["message"]}', file=sys.stderr)
            return ''

    def loop(self, **kwargs) -> None:
        print("直接按 ↵ 可結束對話")
        while True:
            user_msg = input(self._get_prompt())
            if not user_msg.strip(): break # 直接 ↵ 就結束
            if self._process_command(user_msg):
                continue # 指令不需回覆，回頭讓使用者輸入
            text = ''
            display_handle = display(text, display_id=True)
            for reply in self.get_reply_text(
                user_msg,
                tools=self.tools, # 傳入要使用的工具
                **kwargs
            ):
                text += reply
                display_handle.update(Markdown(text))

    def save(self, filename) -> None:
        with open(filename, 'wb') as f:
            pickle.dump(
                {
                    'last_id': self._last_id,
                    'tools': self.tools,
                    'commands': self._commands
                },
                f
            )

    def load(self, filename) -> None:
        with open(filename, 'rb') as f:
            data = pickle.load(f)
            self._last_id = data['last_id']
            self.tools = data['tools']
            self._commands = data['commands']

    def show_thread(self):
        if not self._last_id: return
        inputs = client.responses.input_items.list(self._last_id)
        response = client.responses.retrieve(self._last_id)
        for item in inputs.data[::-1]:
            prompt = ">>> " if item.role == 'user' else ''
            for content in item.content:
                print(f'{prompt}{content.text}')
        print(response.output_text)

    def delete_thread(self):
        if not self._last_id: return
        last_id = self._last_id
        while last_id:
            response = client.responses.retrieve(last_id)
            last_id, curr_id = (
                response.previous_response_id,
                last_id
            )
            client.responses.delete(curr_id)
        self._last_id = None

### 建立處理內建搜尋工具指令的類別


In [ ]:
class WebSearchCommand(BaseComand):
    def __init__(self):
        super().__init__(
            '/w',
            'web_search_preview',
            '🌐'
        )

    def handle_command(self, chat, cmd):
        if not super().handle_command(chat, cmd):
            return False
        idx = chat.find_tool_index(self.tool_name)
        if idx == -1:
            chat.tools.append({
                'type': self.tool_name
            })
        else:
            chat.tools.pop(idx)
        return True

In [ ]:
chat = Chat(client, commands=[WebSearchCommand()])
chat.loop(
    model='gpt-4.1-mini',
    stream=True
)
chat.save('chat.pkl')

In [ ]:
chat1 = Chat(client)
chat1.load('chat.pkl')
chat1.loop(
    model='gpt-4.1-mini',
    stream=True
)

In [ ]:
chat1.delete_thread()

## 4-3 使用內建檔案檢索 RAG 工具

### RAG 簡介

### 上傳檔案進行 RAG

- 《個人資料保護法》網址：https://reurl.cc/g6661X
- 《通訊保障及監察法》網址：https://reurl.cc/5davLv

### 使用內建的檔案檢索工具

In [ ]:
vector_store_id = "vs_68245fe8eff081919422d080954e2225"

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input="在便利商店記下顧客報的會員電話號碼打給他有犯法嗎？",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id]
    }]
)

display(Markdown(response.output_text))

In [ ]:
pprint(response)

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input=[
        {
            'role': 'user',
            'content': [
                {
                    'type': 'input_file',
                    'file_id': "file-WhDZWsm4T5T8PaWFdcPnvt"
                },
                {
                    'type': 'input_text',
                    'text': '在便利商店記下顧客報的會員'\
                            '電話號碼打給他有犯法嗎？'
                }
            ]
        }
    ]
)

display(Markdown(response.output_text))

In [ ]:
pprint(response.usage)

### 查看檢索結果

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input="在便利商店記下顧客報的會員電話號碼打給他有犯法嗎？",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }],
    include=["file_search_call.results"]
)
pprint(response)

In [ ]:
def show_file_search_results(response):
    if response.output[0].type != 'file_search_call': return
    results = response.output[0].results
    if not results: return # 叫用 API 時沒有指定 include 參數
    for i, result in enumerate(results, start=1):
        display(Markdown('---'))
        print(f'{i}. {result.filename}({result.score})')
        display(Markdown('---'))
        display(Markdown(result.text))

show_file_search_results(response)

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input="在便利商店記下顧客報的會員電話號碼打給他有犯法嗎？",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }],
    include=["file_search_call.results"],
    stream=True
)

for event in response:
    pprint(event)

### 限制檢索筆數

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input="在便利商店記下顧客報的會員電話號碼打給他有犯法嗎？",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
        "max_num_results": 3
    }],
    include=["file_search_call.results"]
)

show_file_search_results(response)

In [ ]:
display(Markdown(response.output_text))

In [ ]:
pprint(response.usage)

### 限制相似度

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input="在便利商店記下顧客報的會員電話號碼打給他有犯法嗎？",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
        "ranking_options": {
            "score_threshold": 0.5
        }
    }],
    include=["file_search_call.results"]
)

for result in response.output[0].results:
    print(result.score)

In [ ]:
show_file_search_results(response)

### 檔案檢索工具的計費方式

## 4-4 利用程式碼動態管理要檢索的檔案

In [ ]:
# 無線鍵盤使用手冊
file1_url = "https://coolermaster.egnyte.com/dd/4pPb6Srybx/"
# 電競耳機使用手冊
file2_url = "https://coolermaster.egnyte.com/dd/BtL7gG2IW6/"

Files API 裡面 file 參數可以是字串、位元組序組、或是開啟檔案得到的物件，你可以用 BytesIO 來模擬檔案物件，這樣做的好處就是可以加上檔名，像是：

```python
file = BytesIO(response.content)
file.name = fialename
```

Files API 提供一個簡便的替代方案，可以用 (檔名，檔案內容) 自動幫你建立 BytesIO 物件。


### 動態上傳檔案

In [ ]:
import requests
import os
from io import BytesIO

def upload_file(file_path):
    try:
        if (file_path.startswith('http://')
            or file_path.startswith('https://')):
            response = requests.get(file_path)
            filename = response.headers.get(
                'content-disposition',
                None
            )
            if filename:
                filename = filename.split('filename=')[-1]
                filename = filename.strip('\'"')
            else:
                filename = file_path.split('/')[-1]
            response = client.files.create(
                file=(filename, response.content),
                purpose='user_data'
            )
        else:
            with open(file_path, 'rb') as file:
                response = client.files.create(
                    file=file,
                    purpose='user_data'
                )
    except:
        return None
    return response.id

In [ ]:
file1_id = upload_file(file1_url)
print(file1_id)

### 建立向量儲存區同時加入檔案

In [ ]:
vector_store = client.vector_stores.create(
    name="電腦專家",
    file_ids=[file1_id],
    chunking_strategy={
        'type': 'static', # 預設是 'auto'
        'static': {
            'chunk_overlap_tokens': 400,
            'max_chunk_size_tokens': 800
        }
    }
)

In [ ]:
print(vector_store.id)

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input="我的鍵盤要如何透過藍牙連線？",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id],
        "max_num_results": 3
    }],
    include=["file_search_call.results"]
)

print(response.output_text)

### 動態加入檔案到向量資料庫

In [ ]:
client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=upload_file(file2_url)
)

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input="我耳機上的麥克風可以拆掉嗎？",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id],
        "max_num_results": 3
    }],
    include=["file_search_call.results"]
)

print(response.output_text)

### 顯示向量資料庫內的檔案

In [ ]:
response = client.vector_stores.files.list(vector_store.id)
pprint(response)

In [ ]:
for i, vector_file in enumerate(response.data):
    file = client.files.retrieve(vector_file.id)
    print(f'{i + 1}:{file.filename}')
    print(f'  {vector_file.id}')

### 移除向量儲存區

In [ ]:
def delete_vector_store(vector_store_id):
    response = client.vector_stores.files.list(vector_store_id)
    for vector_file in response.data:
        client.files.delete(vector_file.id)
    client.vector_stores.delete(vector_store_id)

In [ ]:
delete_vector_store(vector_store.id)

## 4-5 幫簡易聊天程式加上檔案檢索功能

### 設計處理檔案檢索指令的類別

In [ ]:
class FileSearchCommand(BaseComand):
    def __init__(self, vector_store_id=None):
        super().__init__(
            '/f',
            'file_search',
            '🔍'
        )
        self.vector_store_id = vector_store_id

    def handle_command(self, chat, cmd):
        if not super().handle_command(chat, cmd):
            return False
        idx = chat.find_tool_index(self.tool_name)
        if len(cmd) < 4: # /f[.]，不是冒號加檔名/網址
            if self.vector_store_id == None:
                print('請先使用 /f:[路徑|網址]上傳檔案')
                return True # 沒有向量資料庫無法切換
            turn_on = (idx == -1) # 切換開/關檔案檢索
        else: # /f:檔名|網址，上傳檔案並開啟檢索功能
            turn_on = True
            file_path = cmd[3:]
            file_id= upload_file(file_path)
            if not file_id:
                print(f'無法上傳檔案：{file_path}')
                return True
            if self.vector_store_id == None:
                vector_store = client.vector_stores.create(
                    name="temp",
                    file_ids=[file_id],
                )
                self.vector_store_id = vector_store.id
            else:
                client.vector_stores.files.create(
                    self.vector_store_id,
                    file_id=file_id
                )
        if turn_on:
            chat.tools.append({
                'type': self.tool_name,
                'vector_store_ids': [self.vector_store_id]
            })
        else:
            chat.tools.pop(idx)
        return True

    def remove_vector_store(self, chat):
        if self.vector_store_id == None: return
        idx = chat.find_tool_index('file_search')
        if idx: chat.tools.pop(idx)
        response = client.vector_stores.files.list(
            self.vector_store_id
        )
        for vector_file in response.data:
            client.files.delete(vector_file.id)
        client.vector_stores.delete(self.vector_store_id)

### 測試具備網頁搜尋與檔案檢索功能的聊天程式

- 無線鍵盤使用手冊：
    https://coolermaster.egnyte.com/dd/4pPb6Srybx/
- 電競耳機使用手冊：
    https://coolermaster.egnyte.com/dd/BtL7gG2IW6/

多工具情況下，都會有偏重檢索檔案的傾向，但是gpt-4.1-mini 明顯比 gpt-4o-mini 好一些，gpt-4.1 似乎也沒有比較厲害：

In [ ]:
file_search_comand = FileSearchCommand()

chat = Chat(
    client,
    commands=[
        file_search_comand,
        WebSearchCommand()
    ],
)
chat.loop(
    model='gpt-4.1-mini',
    stream=True
)
chat.save('test.db')

In [ ]:
chat2 = Chat(client)
chat2.load('test.db')
chat2.tools

In [ ]:
chat2.loop(
    model='gpt-4.1-mini',
    stream=True
)

In [ ]:
file_search_comand.remove_vector_store(chat2)

### 加上可以檢視工具執行結果的功能

In [ ]:
class BaseComand:
    def __init__(self, command, tool_name, icon, verbose=False):
        self.command = command      # 指令
        self.tool_name = tool_name  # 工具名稱
        self.icon = icon            # 工具圖示字元
        self.verbose = verbose      # 啟用詳細輸出
        self.extra_args = {}        # 要額外送給模型的參數

    def handle_command(self, chat, cmd):
        # 不是正確的指令開頭（空字串會是 True）
        if not cmd.startswith(self.command):
            return False
        return True

    def handle_event(self, chat, stream, event):
        return None # 預設不處理交給下一個指令處理器

In [ ]:
class WebSearchCommand(BaseComand):
    def __init__(self, verbose=False):
        super().__init__(
            '/w',
            'web_search_preview',
            '🌐',
            verbose
        )

    def handle_command(self, chat, cmd):
        if not super().handle_command(chat, cmd):
            return False
        idx = chat.find_tool_index(self.tool_name)
        if idx == -1:
            chat.tools.append({
                'type': self.tool_name
            })
        else:
            chat.tools.pop(idx)
        return True

    def show_search_results(self, response):
        if response.output[0].type != "web_search_call":
            return
        content = response.output[1].content[0]
        for i, annotaion in enumerate(
            content.annotations, start=1
        ):
            print(f'{i}. {annotaion.title}')
            print(f'   {unquote(annotaion.url)}')

    def handle_event(self, chat, stream, event):
        if not self.verbose: return None
        if event.type == 'response.completed':
            self.show_search_results(event.response)
        return None

In [ ]:
class FileSearchCommand(BaseComand):
    def __init__(self, vector_store_id=None, verbose=False):
        super().__init__(
            '/f',
            'file_search',
            '🔍',
            verbose
        )
        self.vector_store_id = vector_store_id

    def handle_command(self, chat, cmd):
        if not super().handle_command(chat, cmd):
            return False
        idx = chat.find_tool_index(self.tool_name)
        if len(cmd) < 4: # /f[.]，不是冒號加檔名/網址
            if self.vector_store_id == None:
                print('請先使用 /f:[路徑|網址]上傳檔案')
                return True # 沒有向量資料庫無法切換
            turn_on = (idx == -1) # 切換開/關檔案檢索
        else: # /f:檔名|網址，上傳檔案並開啟檢索功能
            turn_on = True
            file_path = cmd[3:]
            file_id= upload_file(file_path)
            if not file_id:
                print(f'無法上傳檔案：{file_path}')
                return True
            if self.vector_store_id == None:
                vector_store = client.vector_stores.create(
                    name="temp",
                    file_ids=[file_id],
                )
                self.vector_store_id = vector_store.id
            else:
                client.vector_stores.files.create(
                    self.vector_store_id,
                    file_id=file_id
                )
        if turn_on:
            chat.tools.append({
                'type': self.tool_name,
                'vector_store_ids': [self.vector_store_id]
            })
            self.extra_args = {
                'include': ['file_search_call.results']
            }
        else:
            chat.tools.pop(idx)
            self.extra_args = {}
        return True

    def remove_vector_store(self, chat):
        if self.vector_store_id == None: return
        idx = chat.find_tool_index('file_search')
        if idx: chat.tools.pop(idx)
        response = client.vector_stores.files.list(
            self.vector_store_id
        )
        for vector_file in response.data:
            client.files.delete(vector_file.id)
        client.vector_stores.delete(self.vector_store_id)

    def show_file_search_results(self, response):
        if response.output[0].type != 'file_search_call':
            return
        results = response.output[0].results
        if not results: return
        for i, result in enumerate(results, start=1):
            display(Markdown('---'))
            print(f'{i}. {result.filename}({result.score})')
            display(Markdown('---'))
            display(Markdown(result.text))

    def handle_event(self, chat, stream, event):
        if not self.verbose: return None
        if event.type == 'response.completed':
            self.show_file_search_results(event.response)
        return None

In [ ]:
class Chat:
    def __init__(self, client, **kwargs):
        self._client = client
        self._last_id = kwargs.pop('last_id', None)
        self.tools = [] # 預設沒有使用工具
        self._commands = kwargs.pop('commands', [])

    def find_tool_index(self, tool_type):
        for i, tool in enumerate(self.tools):
            if tool['type'] == tool_type: return i
        return -1

    def _get_prompt(self):
        prompt = ''
        for command in self._commands:
            idx = self.find_tool_index(command.tool_name)
            if idx != -1:
                prompt += f'{command.icon}'
        prompt = f'({prompt})>>> ' if prompt else '>>> '
        return prompt

    def _process_command(self, cmd):
        for command in self._commands:
            if command.handle_command(self, cmd):
                return True
        return False

    def get_reply_text(self, msg, **kwargs) -> str:
        instructions = kwargs.pop('instructions', '使用繁體中文')
        model = kwargs.pop('model', 'gpt-4.1-nano')
        stream = kwargs.pop('stream', False)
        for command in self._commands:
            kwargs.update(command.extra_args)
        try:
            response = self._client.responses.create(
                instructions=instructions,
                model=model,
                input=msg,
                stream=True, # 都以串流方式處理，簡化程式邏輯
                previous_response_id=self._last_id, # 串接回應
                **kwargs
            )
            for event in response:
                for command in self._commands:
                    result = command.handle_event(
                        self, stream, event
                    )
                    if result: break
                if event.type == 'response.output_text.delta':
                    if stream: # 串流模式生成片段內容
                        yield event.delta
                elif event.type == 'response.completed':
                    self._last_id = event.response.id # 記錄識別碼
                    if not stream: # 非串流生成完整內容
                        yield event.response.output_text
        except openai.APIError as err:
            print(f'Error:{err.body["message"]}', file=sys.stderr)
            return ''

    def loop(self, **kwargs) -> None:
        print("直接按 ↵ 可結束對話")
        while True:
            user_msg = input(self._get_prompt())
            if not user_msg.strip(): break # 直接 ↵ 就結束
            if self._process_command(user_msg):
                continue # 指令不需回覆，回頭讓使用者輸入
            text = ''
            display_handle = display(text, display_id=True)
            for reply in self.get_reply_text(
                user_msg,
                tools=self.tools, # 傳入要使用的工具
                **kwargs
            ):
                text += reply
                display_handle.update(Markdown(text))

    def save(self, filename) -> None:
        with open(filename, 'wb') as f:
            pickle.dump(
                {
                    'last_id': self._last_id,
                    'tools': self.tools,
                    'commands': self._commands
                },
                f
            )

    def load(self, filename) -> None:
        with open(filename, 'rb') as f:
            data = pickle.load(f)
            self._last_id = data['last_id']
            self.tools = data['tools']
            self._commands = data['commands']

    def show_thread(self):
        if not self._last_id: return
        inputs = client.responses.input_items.list(self._last_id)
        response = client.responses.retrieve(self._last_id)
        for item in inputs.data[::-1]:
            prompt = ">>> " if item.role == 'user' else ''
            for content in item.content:
                print(f'{prompt}{content.text}')
        print(response.output_text)

    def delete_thread(self):
        if not self._last_id: return
        last_id = self._last_id
        while last_id:
            response = client.responses.retrieve(last_id)
            last_id, curr_id = (
                response.previous_response_id,
                last_id
            )
            client.responses.delete(curr_id)
        self._last_id = None

In [ ]:
file_search_comand = FileSearchCommand(verbose=False)

chat = Chat(
    client,
    commands=[
        file_search_comand,
        WebSearchCommand(verbose=True)
    ]
)
chat.loop(
    model='gpt-4.1-mini',
    stream=True,
)

In [ ]:
chat.delete_thread()
file_search_comand.remove_vector_store(chat)